In [1]:
import os, sys
import numpy as np
import h5netcdf

import xarray as xr
import xarray.ufuncs as xu
import pandas as pd

from matplotlib import pyplot as plt

import cartopy.crs as ccrs
import cartopy

plt.rc("figure", figsize=(16,4))
plt.rc("font", size=14)
ccrs.PlateCarree()

from dask.distributed import Client, LocalCluster
cluster = LocalCluster(processes=False, n_workers=1, threads_per_worker=4)
client = Client(cluster)
client



Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://137.129.155.67:8787/status,
Dashboard: http://137.129.155.67:8787/status,Workers: 1
Total threads: 4,Total memory: 78.61 GiB
Status: running,Using processes: False
Comm: inproc://137.129.155.67/61720/1,Workers: 1
Dashboard: http://137.129.155.67:8787/status,Total threads: 4
Started: Just now,Total memory: 78.61 GiB
Comm: inproc://137.129.155.67/61720/4,Total threads: 4
Dashboard: http://137.129.155.67:41465/status,Memory: 78.61 GiB
Nanny: None,


In [3]:
indir_data_FILTERED = '/cnrm/tropics/commun/DATACOMMUN/WAVE/NO_SAVE/DATA/FILTERED_ANOMALY/'
filenames = np.arange(2001,2019)

# datasets = []
# for f in filenames:
ds = xr.open_mfdataset(indir_data_FILTERED + 'TCWV/*.nc', chunks = {'time' : 365}, parallel = True)
#     datasets.append(ds)
# ds_anom_OLR = xr.concat(datasets, dim='time', coords='minimal', compat='override')
# _ds_anom_OLR = ds_anom_OLR.assign_coords(lon=(((ds_anom_OLR.lon + 180) % 360) - 180)).sortby('lon')
# _ds_anom_OLR = _ds_anom_OLR.sel(lon = slice(-30,30))
ds = ds.sel(lat = slice(30,-30))
ds

<xarray.Dataset>
Dimensions:      (time: 84736, lat: 241, lon: 1440)
Coordinates:
  * time         (time) datetime64[ns] 1991-01-01 ... 2019-12-31T21:00:00
  * lat          (lat) float64 30.0 29.75 29.5 29.25 ... -29.5 -29.75 -30.0
  * lon          (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.2 359.5 359.8
Data variables:
    TCWV_Kelvin  (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>
    TCWV_Rossby  (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>
    TCWV_MJO     (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>
    TCWV_MRG     (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>
    TCWV_EIG     (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>
    TCWV_WIG     (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>
    TCWV_TD      (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>
    TCWV_LF      (time, lat, lon) float64 dask.array<chunksize=(365, 241, 1440), meta=np.ndarray>

In [4]:
ds_Var = ds.var(dim = 'time')
ds_Var = ds_Var

In [5]:
map_proj = ccrs.Mercator(central_longitude=0.0, 
                         min_latitude=-30.0, 
                         max_latitude=30.0,
                         globe=None)

# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = -10, +10

In [6]:
# plt.rc("figure", figsize=(20,14))
cmap ='BrBG'
vmin, vmax = -10, +10

#####################################################################################
colorbar = {'label': 'OLR $W.m^{-2}$',
            'orientation': 'vertical' ,
            'extend' : 'both'}

fig, axis = plt.subplots( 1, 1, 
    figsize=(40,20), subplot_kw={'projection': map_proj})

ds_Var['TCWV_Kelvin'].plot.contourf(ax=axis, levels = 101, vmin = vmin, vmax = vmax,
                    transform=ccrs.PlateCarree(),
                    cmap=cmap ,
                    cbar_kwargs=colorbar)

axis.coastlines()
gl = axis.gridlines(draw_labels=True)
gl.top_labels = False
gl.right_labels = False
plt.show()

KeyboardInterrupt: 

Error in callback <function flush_figures at 0x7fe7f93f1dc0> (for post_execute):


Exception ignored in: <function WeakSet.__init__.<locals>._remove at 0x7fe88a3bd3a0>
Traceback (most recent call last):
  File "/home/durandy/miniconda3/envs/towel/lib/python3.9/_weakrefset.py", line 39, in _remove
    def _remove(item, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
ds_V = ds_anom_OLR*0
i = 0
for f in filenames:
    ds = xr.open_mfdataset(indir_data_FILTERED + 'OLR/*CERES*' + str(f) + '*.nc')
    ds_V, i = ds_V + ds, i + 1
ds_V = ds_V/i
ds_V